In [31]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import os

import time #cpu time
import psutil #memory usage

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve

# For Visualization
import matplotlib.pyplot as plt
#displays better in jupyter notebooks
%matplotlib inline

In [32]:
def get_data(file):
    data = load_svmlight_file(file)#avazu-app.tr.bz2
    return data[0], data[1]

In [33]:
#!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a1a.t

In [34]:
mem_baseline=psutil.virtual_memory() #  physical memory usage
print('Here is the memory baseline before importing data:\n',mem_baseline)

Here is the memory baseline before importing data:
 svmem(total=17101512704, available=9249325056, percent=45.9, used=7852187648, free=9249325056)


In [35]:
X, y =get_data('a1a.t') #import raw data
mem_InData=psutil.virtual_memory()

In [36]:
print('Here is the memory usage after importing data:\n',mem_InData) #  physical memory usage
print('\nThe time took to import the raw data:')
exec_time1 = %%timeit -o X, y =get_data('a1a.t') #import raw data

Here is the memory usage after importing data:
 svmem(total=17101512704, available=9250664448, percent=45.9, used=7850848256, free=9250664448)

The time took to import the raw data:
541 ms ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
#%%timeit
#X, y =get_data('a1a.t.bz2') #import binary data

In [38]:
print('Here is the sparse matrix format for the data:') 
X

Here is the sparse matrix format for the data:


<30956x123 sparse matrix of type '<class 'numpy.float64'>'
	with 429343 stored elements in Compressed Sparse Row format>

In [39]:
#May not be appropriate for time dependant data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [40]:
# instantiate a logistic regression model, and fit with X and y
# Here are the 
model = linear_model.LogisticRegression(penalty='l2',\
                                        C=1.0,\
                                        tol=0.0001,\
                                        fit_intercept=True,\
                                        max_iter=100)
start = time.time()
model = model.fit(X, y.ravel())
end = time.time()

In [41]:
exec_time2=(end - start)
print('The time taken to execute the logistic regression:',exec_time2,'seconds')

The time taken to execute the logistic regression: 1.0989973545074463 seconds


In [42]:
y_pred = model.predict(X_test)
y_obs = y_test
y_score = y_pred

In [43]:
r2=metrics.r2_score(y_obs, y_pred)
accuracy=model.score(X_train, y_train)
prec=metrics.precision_score(y_obs, y_pred, labels=None, pos_label=1)
recall=metrics.recall_score(y_obs, y_pred, labels=None, pos_label=1)
f1 = metrics.f1_score(y_obs,y_pred)
ROC_AUC = metrics.roc_auc_score(y_obs, y_score)
print('The correlation coefficient:',r2,\
      '\nThe accuracy of the model:',accuracy,\
      '\nThe precision (tp / (tp + fp)):',prec,\
      '\nThe recall (tp / (tp + fn)):',recall,\
      '\nThe f1 score is:',f1,\
      '\nThe Area Under the Curve score is:',ROC_AUC)

The correlation coefficient: 0.1588200017358622 
The accuracy of the model: 0.8501928640308583 
The precision (tp / (tp + fp)): 0.7191295546558705 
The recall (tp / (tp + fn)): 0.5874328234807772 
The f1 score is: 0.6466439135381115 
The Area Under the Curve score is: 0.7581257999666293
